In [1]:
import sys
sys.path.append("../baseline_models")

In [2]:
import numpy as np
import xarray as xr
import pandas as pd

import matplotlib.pyplot as plt
import glob


In [3]:
from utils import data_path

# This doesn't work with the NorESM CMIP data for some reason
_Y = xr.open_dataset(data_path + 'outputs_ssp245.nc')

weights = np.cos(np.deg2rad(_Y['tas'].lat)).expand_dims(lon=144).assign_coords(lon=_Y.lon)

def global_mean(da):
    return da.weighted(weights).mean(['lat', 'lon'])

weights

<xarray.DataArray 'lat' (lon: 144, lat: 96)>
array([[6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17],
       [6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17],
       [6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17],
       ...,
       [6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17],
       [6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17],
       [6.12323400e-17, 3.30633693e-02, 6.60905843e-02, ...,
        6.60905843e-02, 3.30633693e-02, 6.12323400e-17]])
Coordinates:
  * lat       (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    quantile  float64 ...
  * lon       (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    units:    degrees

In [4]:
model_labels = [
 'CNRM-CM6-1',
 'ACCESS-CM2',
 'INM-CM5-0',
 'MIROC6',
 'HadGEM3-GC31-LL',
 'MRI-ESM2-0',
 'MCM-UA-1-0',
 'FGOALS-g3',
 'CMCC-CM2-SR5',
 'CanESM5',
 'EC-Earth3',
 'INM-CM4-8',
 'BCC-CSM2-MR',
 'CESM2',
 'MPI-ESM1-2-LR',
 'CESM2-WACCM',
 'NorESM2-MM',
 'CNRM-CM6-1-HR',
 'MIROC-ES2L',
 'EC-Earth3-Veg',
 'CNRM-ESM2-1'
]

In [5]:
# Example NorESM grid
noresm = xr.open_mfdataset("../CMIP6/*_Ayr_NorESM2-LM_ssp245_r1i1p1f1_gn_2015-2100.nc").interp_like(weights)
noresm['diurnal_temperature_range'] = noresm['tasmax'] - noresm['tasmin']
# noresm = noresm.drop('time').assign_coords(year=noresm['time'].dt.year)
noresm.coords['time'] = noresm['time'].dt.year
noresm

<xarray.Dataset>
Dimensions:                    (time: 86, lat: 96, lon: 144)
Coordinates:
  * lat                        (lat) float64 -90.0 -88.11 -86.21 ... 88.11 90.0
  * lon                        (lon) float64 0.0 2.5 5.0 ... 352.5 355.0 357.5
  * time                       (time) int64 2015 2016 2017 ... 2098 2099 2100
Data variables:
    pr                         (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    tas                        (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    tasmax                     (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    tasmin                     (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    diurnal_temperature_range  (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
Attributes: (12/42)
    description:            Trying global attribute
    source_id:              NorESM2-LM
    brol:                   Trying global attribute
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          Hybrid-restart from year 2015-01-01 of historical
    ...                     ...
    sub_experiment_id:      none
    table_id:               Amon
    variable_id:            pr
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by NCC is licensed unde...
    tracking_id:            hdl:21.14100/4bcaa71a-90fb-4eda-bc3f-4ce0f2009a3c

In [9]:
models = []
for model in model_labels:
    print(model)
    try:
        cmip_ds = xr.open_mfdataset(f"../CMIP6/*_Ayr_{model}_ssp245_*.nc", decode_times=True)
        piclim = xr.open_mfdataset(f"../CMIP6/*_Ayr_{model}_piControl_*.nc", decode_times=True).isel(time=slice(-85, None)).mean('time')
        cmip_ds = cmip_ds - piclim
    #     print(cmip_ds['time'].dt.year)
        cmip_ds.coords['time'] = cmip_ds['time'].dt.year
#     print(cmip_ds['year'])
        if 'tasmin' in cmip_ds:
            cmip_ds['diurnal_temperature_range'] = cmip_ds['tasmax'] - cmip_ds['tasmin']
        else:
            cmip_ds['diurnal_temperature_range'] = xr.full_like(cmip_ds['tas'], np.NaN)
        models.append(cmip_ds.interp_like(weights))
    except Exception:
        print("Failed to read model")

CNRM-CM6-1
ACCESS-CM2
INM-CM5-0


/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:167: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/indexing.py:560: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason

MIROC6
HadGEM3-GC31-LL


/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/indexing.py:560: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/

MRI-ESM2-0
MCM-UA-1-0
FGOALS-g3
CMCC-CM2-SR5
CanESM5
EC-Earth3


/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/indexing.py:560: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/

INM-CM4-8
BCC-CSM2-MR
CESM2
MPI-ESM1-2-LR
CESM2-WACCM


/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/conventions.py:428: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/conventions.py:428: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/conventions.py:428: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/conventions.py:428: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/conventions.py:428: Serializa

NorESM2-MM
CNRM-CM6-1-HR
MIROC-ES2L
EC-Earth3-Veg
CNRM-ESM2-1


/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/indexing.py:560: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/coding/times.py:831: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/root/miniconda3/envs/newenv/lib/python3.10/site-packages/xarray/core/

In [10]:
models[2]

<xarray.Dataset>
Dimensions:                    (time: 86, lat: 96, lon: 144)
Coordinates:
  * time                       (time) int64 2015 2016 2017 ... 2098 2099 2100
  * lat                        (lat) float64 -90.0 -88.11 -86.21 ... 88.11 90.0
  * lon                        (lon) float64 0.0 2.5 5.0 ... 352.5 355.0 357.5
Data variables:
    pr                         (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    tas                        (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>
    diurnal_temperature_range  (time, lat, lon) float32 dask.array<chunksize=(86, 96, 144), meta=np.ndarray>

In [11]:
Y = xr.concat([model.assign_coords(model=label) for model, label in zip(models, model_labels)], 'model').compute()

In [12]:
variables = ['tas', 'pr', 'diurnal_temperature_range']

R2E = pd.DataFrame({
    "Variability": {variable: global_mean(Y[variable].sel(time=slice(2080, None)).mean('time')).std().data / np.abs(global_mean(Y.mean('model')[variable].sel(time=slice(2080, None)).mean('time')).data) for variable in variables} 
})

NRMSE = pd.DataFrame({
    "Variability": {variable: global_mean(Y[variable].sel(time=slice(2080, None)).mean('time').std('model')).data/ np.abs(global_mean(Y.mean('model')[variable].sel(time=slice(2080, None)).mean('time')).data) for variable in variables}
})

variability_df = pd.concat([NRMSE, R2E, NRMSE+5*R2E], keys=['Spatial', 'Global', 'Total']).T.swaplevel(axis=1)[variables]



In [13]:
variability_df

tas                            pr                     \
              Spatial    Global     Total   Spatial   Global     Total   
Variability  0.258005  0.176679  1.141402  1.993657  0.38922  3.939755   

            diurnal_temperature_range                     
                              Spatial   Global     Total  
Variability                  1.962778  0.79911  5.958327